# Guide
실행 불필요  
전처리 완료된 데이터 링크 아래 참고

---

# Data
[[Google Drive] /Data/Final/Step2-2.zip](https://drive.google.com/file/d/19ARNrbgWSn-Wi5k7uXKehJnzmvbIRThv/view?usp=sharing)

## Dataset Source
- [[Kaggle] Fruit and Vegetable Disease (Healthy vs Rotten)](https://www.kaggle.com/datasets/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten)  
apple, banana, mango, orange, pomegranate, strawberry, tomato 데이터만 사용  
- [[Roboflow] apples Computer Vision Project](https://universe.roboflow.com/ds-lxa2d/apples-daz2v)  
apple 데이터 추가  
- [[Kaggle] Lemon Quality Dataset](https://www.kaggle.com/datasets/yusufemir/lemon-quality-dataset)  
lemon 데이터 사용  
- [[AI Hub] 국내 재배 아열대·열대 병해충 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71750)  
olive 데이터 사용
- [[Roboflow] PomegranateRotten Computer Vision Project](https://universe.roboflow.com/uit-90j5o/pomegranaterotten)  
[[Roboflow] G2POMOGRANATE Computer Vision Project](https://universe.roboflow.com/ratul/g2pomogranate)  
[[Roboflow] Pomegranate disease Computer Vision Project](https://universe.roboflow.com/facedetection-keut1/pomegranate-disease)  
Pomegranate_Rotten 데이터 추가  

## Preprocessing
- Kaggle  
기존 데이터셋에는 이미 증강 이미지 존재(일부 클래스에만) → 다른 Step의 모델에도 사용하기 위해 증강 이미지 삭제 + 중복 이미지 삭제  
- Kaggle(lemon), AI Hub, Roboflow  
이미지 중 모델 학습에 적합하다고 판단되는 데이터만 사용 + 중복 이미지 삭제

---

# Data Random Undersampling
## Data
|              |&nbsp;&nbsp;&nbsp;Original&nbsp;&nbsp;&nbsp;| Remove |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Result&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 300 | 0 | 300 |
| Apple_Rotten | 300 | 0 | 300 | 
| Banana_Fresh | 300 | 0 | 300 |
| Banana_Rotten | 300 | 0 | 300 |
| Lemon_Fresh  | 1125 | 825 | 300 |
| Lemon_Rotten | 951 | 651 | 300 |
| Mango_Fresh  | 528 | 228 | 300 |
| Mango_Rotten | 930 | 630 | 300 |
| Olive_Fresh  | 800 | 500 | 300 |
| Olive_Rotten | 800 | 500 | 300 |
| Orange_Fresh | 300 | 0 | 300 |
| Orange_Rotten | 300 | 0 | 300 |
| Pomegranate_Fresh  | 317 | 17 | 300 |
| Pomegranate_Rotten | 292 | 0 | 300 |
| Strawberry_Fresh  | 1346 | 1046 | 300 |
| Strawberry_Rotten | 1252 | 952 | 300 |
| Tomato_Fresh  | 595 | 295 | 300 |
| Tomato_Rotten | 573 | 273 | 300 |


데이터가 가장 적은 Pomegranate_Rotten의 수(300)로 나머지 클래스 모두 언더샘플링  
apple, banana, orange : Step2-1에서 사용한 데이터 그대로 사용

In [1]:
import os
import random

In [2]:
def random_undersampling(directory):

    image_files = [f for f in os.listdir(directory) if f.endswith((".jpg", ".png"))]
    
    delete_count = len(image_files) - 300

    # 삭제할 파일 랜덤 선택
    files_to_delete = random.sample(image_files, delete_count)

    # 삭제된 파일 수
    count = 0

    # 파일 삭제
    for file_name in files_to_delete:
        file_path = os.path.join(directory, file_name)
        os.remove(file_path)
    
        count += 1
    
    print(f"deleted images: {count}")
    print(f"current images: {len([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])}")

In [3]:
base_dir = "/tf/Fixed_Data/Data_Final/Data"

In [4]:
# apple, banana, orange 데이터는 Step2-1의 데이터 사용
classes = ['Lemon_Fresh',
           'Lemon_Rotten',
           'Mango_Fresh',
           'Mango_Rotten',
           'Olive_Fresh',
           'Olive_Rotten',
           'Pomegranate_Fresh',
           'Pomegranate_Rotten',
           'Strawberry_Fresh',
           'Strawberry_Rotten',
           'Tomato_Fresh',
           'Tomato_Rotten']

names = ['lemon', 'mango', 'olive', 'pomegranate', 'strawberry', 'tomato']

In [5]:
for i in range(len(classes)):
    print(classes[i])
    dir = os.path.join(base_dir, names[i // 2])
    
    freshness = 'fresh' if i % 2 == 0 else 'stale'
    dir = os.path.join(dir, freshness)
    
    random_undersampling(dir)
    print()

Lemon_Fresh
deleted images: 825
current images: 300

Lemon_Rotten
deleted images: 651
current images: 300

Mango_Fresh
deleted images: 228
current images: 300

Mango_Rotten
deleted images: 630
current images: 300

Olive_Fresh
deleted images: 500
current images: 300

Olive_Rotten
deleted images: 500
current images: 300

Pomegranate_Fresh
deleted images: 17
current images: 300

Pomegranate_Rotten
deleted images: 0
current images: 300

Strawberry_Fresh
deleted images: 1046
current images: 300

Strawberry_Rotten
deleted images: 952
current images: 300

Tomato_Fresh
deleted images: 295
current images: 300

Tomato_Rotten
deleted images: 273
current images: 300



# Data Augmentation
`ImageDataGenerator` 사용

## Augmentation Ratio
1개의 이미지 당 2개의 증강 이미지 생성

## Data
|              |&nbsp;&nbsp;&nbsp;Original&nbsp;&nbsp;&nbsp;| Augmentation |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Final&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 900 | 0 | 900 |
| Apple_Rotten | 900 | 0 | 900 |
| Banana_Fresh | 900 | 0 | 900 |
| Banana_Rotten | 900 | 0 | 900 |
| Lemon_Fresh  | 300 | 600 | 900 |
| Lemon_Rotten | 300 | 600 | 900 |
| Mango_Fresh  | 300 | 600 | 900 |
| Mango_Rotten | 300 | 600 | 900 |
| Olive_Fresh  | 300 | 600 | 900 |
| Olive_Rotten | 300 | 600 | 900 |
| Orange_Fresh | 900 | 0 | 900 |
| Orange_Rotten | 900 | 0 | 900 |
| Pomegranate_Fresh  | 300 | 600 | 900 |
| Pomegranate_Rotten | 300 | 600 | 900 |
| Strawberry_Fresh  | 300 | 600 | 900 |
| Strawberry_Rotten | 300 | 600 | 900 |
| Tomato_Fresh  | 300 | 600 | 900 |
| Tomato_Rotten | 300 | 600 | 900 |

apple, banana, orange : Step2-1에서 사용한 데이터 그대로 사용

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import uuid

2024-11-21 17:12:12.279237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
datagen = ImageDataGenerator(
    rotation_range=90,            # 회전 각도 범위 [-90, +90]
    shear_range=0.3,              # 전단 변형(층밀림) 각도 [0, 0.3](radian):최대 약 17도
    zoom_range=[0.7, 1.3],        # 확대/축소 범위. 원본의 70% ~ 130%
    brightness_range=[0.2, 1.3],  # 밝기 조정 범위. 원본의 20% ~ 130%
    horizontal_flip=True,         # 좌우 반전
    vertical_flip=True,           # 상하 반전
)

In [8]:
def data_augmentation(input_dir, output_dir):
    
    for filename in os.listdir(input_dir):
        
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            img_path = os.path.join(input_dir, filename)
            img = load_img(img_path)  # 이미지 로드
            x = img_to_array(img)     # numpy 배열로 변환
            x = np.expand_dims(x, axis=0)  # 배치 차원 추가
            
            # 원본 이미지 저장
            original_output_path = os.path.join(output_dir, f'original_{filename}')
            img.save(original_output_path)  # 원본 이미지 저장
        
            # 증강 이미지 생성 및 저장
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=f'aug_{filename.split(".")[0]}_{str(uuid.uuid4())}', save_format='jpg'):
                i += 1
                if i >= 2:  # 하나의 이미지당 2개의 증강 이미지
                    break

In [9]:
for i in range(len(classes)):
    
    input_dir = os.path.join(base_dir, names[i // 2])
    
    freshness = 'fresh' if i % 2 == 0 else 'stale'
    input_dir = os.path.join(input_dir, freshness)
    
    output_dir = os.path.join(base_dir, 'Step2', classes[i])
    
    data_augmentation(input_dir, output_dir)

# Data Split
## Original and Augmented Data
/tf/Fixed_Data/Data_Final/Data/Step2  
│  
├── Apple_Fresh  
├── Apple_Rotten  
├── Banana_Fresh  
├── Banana_Rotten  
├── Lemon_Fresh  
├── Lemon_Rotten  
├── Mango_Fresh  
├── Mango_Rotten  
├── Olive_Fresh  
├── Olive_Rotten  
├── Orange_Fresh  
├── Orange_Rotten  
├── Pomegranate_Fresh  
├── Pomegranate_Rotten  
├── Strawberry_Fresh  
├── Strawberry_Rotten  
├── Tomato_Fresh  
└── Tomato_Rotten  

## Splited Data
/tf/Fixed_Data/Data_Final/Step2   
│  
├── train  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Rotten  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Banana_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Tomato_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Tomato_Rotten  
│  
├── validation  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Rotten  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Banana_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Tomato_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Tomato_Rotten  
│  
└── test  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Rotten  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Banana_Fresh  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── ...  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Tomato_Fresh  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Tomato_Rotten  

    
## Split Ratio
train : validation : test = 7 : 2 : 1

## Dataset
|              | &nbsp;&nbsp;&nbsp; train &nbsp;&nbsp;&nbsp; | validation | &nbsp;&nbsp;&nbsp;&nbsp; test &nbsp;&nbsp;&nbsp;&nbsp; |
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 630 | 180 | 90 |
| Apple_Rotten | 630 | 180 | 90 |
| Banana_Fresh  | 630 | 180 | 90 |
| Banana_Rotten | 630 | 180 | 90 |
| Lemon_Fresh  | 630 | 180 | 90 |
| Lemon_Rotten | 630 | 180 | 90 |
| Mango_Fresh  | 630 | 180 | 90 |
| Mango_Rotten | 630 | 180 | 90 |
| Olive_Fresh  | 630 | 180 | 90 |
| Olive_Rotten | 630 | 180 | 90 |
| Orange_Fresh | 630 | 180 | 90 |
| Orange_Rotten | 630 | 180 | 90 |
| Pomegranate_Fresh  | 630 | 180 | 90 |
| Pomegranate_Rotten | 630 | 180 | 90 |
| Strawberry_Fresh  | 630 | 180 | 90 |
| Strawberry_Rotten | 630 | 180 | 90 |
| Tomato_Fresh  | 630 | 180 | 90 |
| Tomato_Rotten | 630 | 180 | 90 |
| Total | 11340 | 3240 | 1620 |

## Code
교재 Chapter 8-2 서브셋 저장 코드 참고


In [10]:
import shutil
import random

### Source : 교재 코드 8-6

In [11]:
# 파일 리스트를 서브셋으로 저장
def make_subset(subset_name, file_list):
    
    for file_path, category in file_list:
        
        dst_dir = os.path.join(new_base_dir, subset_name, category)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(file_path, os.path.join(dst_dir, os.path.basename(file_path)))

In [12]:
# 주어진 비율로 데이터 분할 후 서브셋 생성
def split_data(base_dir, train_ratio, val_ratio):
    
    # 클래스 디렉토리 수집
    class_dirs = [d for d in os.listdir(base_dir) 
                  if os.path.isdir(os.path.join(base_dir, d)) and not d.startswith('.')]
    
    train_data = 0
    val_data = 0
    test_data = 0
    
    # 데이터 분할
    for class_dir in class_dirs:
        
        class_path = os.path.join(base_dir, class_dir)
        
        # 파일 목록 수집
        files = [(os.path.join(class_path, f), class_dir) 
                 for f in os.listdir(class_path) if not f.startswith('.')]
        
        # 데이터 shuffle
        random.shuffle(files)

        # 데이터 분할
        total = len(files)
        train_index = int(total * train_ratio)
        val_index = train_index + int(total * val_ratio)

        train_files = files[:train_index]
        val_files = files[train_index:val_index]
        test_files = files[val_index:]

        # 서브셋 생성
        make_subset("train", train_files)
        make_subset("validation", val_files)
        make_subset("test", test_files)
        
        train_data += len(train_files)
        val_data += len(val_files)
        test_data += len(test_files)

    return train_data, val_data, test_data

In [13]:
original_dir = "/tf/Fixed_Data/Data_Final/Data/Step2"
new_base_dir = "/tf/Fixed_Data/Data_Final/Step2-2"

In [14]:
# train:validation:test = 7:2:1
train_ratio = 0.7
val_ratio = 0.2

In [15]:
# 데이터 분할 및 서브셋 생성
train_count, val_count, test_count = split_data(original_dir, train_ratio, val_ratio)

print(f"Train files: {train_count}, Validation files: {val_count}, Test files: {test_count}")

Train files: 11340, Validation files: 3240, Test files: 1620
